# Importing Libraries

In [1]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten,Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
import numpy as np
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt

In [2]:
def resizing(img_arr):
    # get image dimensions
    height, width, color = img_arr.shape

    # crop border 20 from left/right and 50 from top/bottom
    x1 = [20, 50]  # upper left hand crop point
    y1 = [width - 20, height - 50]  # lower right hand crop point

    crop1 = img_arr[x1[1]:y1[1], x1[0]:y1[0], 0:color]

    # crop to square
    height, width, color = crop1.shape  # new cropped dimensions

    if width > height:
        diff = int((width - height)/2)
        x2 = [diff, 0]  # upper left hand crop point
        y2 = [width - diff, height]  # bottom right hand crop point
        crop2 = crop1[x2[1]:y2[1], x2[0]:y2[0], 0:color]  # creates cropped square image
    elif height > width:
        diff = int((height - width)/2)
        x2 = [0, diff]  # upper left hand crop point
        y2 = [width, height - diff]  # bottom right hand crop point
        crop2 = crop1[x2[1]:y2[1], x2[0]:y2[0], 0:color]  # creates cropped square image
    else:
        crop2 = crop1

    # ensure image is rgb
    if crop2.shape[2] == 3:
        img_arr = crop2  # no need to convert if image is already rgb
    else:
        img_arr = cv2.cvtColor(crop2,cv2.COLOR_GRAY2RGB)  # convert to rgb if image is grayscale
    return img_arr

In [3]:
train_path="all_updated2/train"
test_path="all_updated2/test"
val_path="all_updated2/val"

# Data Preprocessing
Includes cropping, resizing, and grayscaling data

In [4]:
x_train=[]
for folder in os.listdir(train_path):
    if (folder != ".DS_Store"):
        sub_path=train_path+"/"+folder
        for img in os.listdir(sub_path):
            image_path=sub_path+"/"+img
            img_arr=cv2.imread(image_path)
            if img_arr is None:
                continue
            else:
                img_arr = resizing(img_arr)
                img_arr=cv2.resize(img_arr,(224,224))
                x_train.append(img_arr)

x_test=[]
for folder in os.listdir(test_path):
    if (folder != ".DS_Store"):
        sub_path=test_path+"/"+folder
        for img in os.listdir(sub_path):
            image_path=sub_path+"/"+img
            img_arr=cv2.imread(image_path)
            if img_arr is None:
                continue
            else:
                img_arr = resizing(img_arr)
                img_arr=cv2.resize(img_arr,(224,224))
                x_test.append(img_arr)
                
x_val=[]
for folder in os.listdir(val_path):
    if (folder != ".DS_Store"):
        sub_path=val_path+"/"+folder
        for img in os.listdir(sub_path):
            image_path=sub_path+"/"+img
            img_arr=cv2.imread(image_path)
            if img_arr is None:
                continue
            else:
                img_arr = resizing(img_arr)
                img_arr=cv2.resize(img_arr,(224,224))
                x_val.append(img_arr)
                

# Normalize Data Arrays

In [5]:
train_x=np.array(x_train)/255.0
test_x=np.array(x_test)/255.0
val_x=np.array(x_val)/255.0


In [6]:
# Compute labels of corresponding datasets
train_datagen = ImageDataGenerator(rescale = 1./255)
test_datagen = ImageDataGenerator(rescale = 1./255)
val_datagen = ImageDataGenerator(rescale = 1./255)

# generate batches of augmented data
training_set = train_datagen.flow_from_directory(train_path,
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'sparse')
test_set = test_datagen.flow_from_directory(test_path,
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'sparse')
val_set = val_datagen.flow_from_directory(val_path,
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'sparse')

Found 481 images belonging to 3 classes.
Found 162 images belonging to 3 classes.
Found 158 images belonging to 3 classes.


In [7]:
train_y=training_set.classes
test_y=test_set.classes
val_y=val_set.classes

In [8]:
training_set.class_indices
train_y.shape,test_y.shape,val_y.shape

((481,), (162,), (158,))

# Model Building
We will be using the VGG-19 pre-trained model to classify ap, ie, and mvvd.

In [9]:
vgg = VGG19(input_shape=(224, 224, 3), weights='imagenet', include_top=False)

# We will not train the pre-trained layers
for layer in vgg.layers:
    layer.trainable = False

2022-03-27 22:06:44.689023: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [10]:
x = Flatten()(vgg.output)
#adding output layer.Softmax classifier is used as it is multi-class classification
prediction = Dense(3, activation='softmax')(x)

model = Model(inputs=vgg.input, outputs=prediction)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [11]:
# Compile the model
model.compile(
  loss='sparse_categorical_crossentropy',
  optimizer="adam",
  metrics=['accuracy']
)

# Model Fitting

To avoid overfitting, we will do an early stop if the loss substantially increases

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop=EarlyStopping(monitor='val_loss',mode='min',verbose=1,patience=5)

history = model.fit(
  train_x,
  train_y,
  validation_data=(val_x,val_y),
  epochs=10,
  callbacks=[early_stop],
  batch_size=32,shuffle=True)

2022-03-27 22:06:45.353335: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/10
16/16 [==============================] - 129s 8s/step - loss: 2.4421 - accuracy: 0.3659 - val_loss: 2.4586 - val_accuracy: 0.4177
Epoch 2/10
16/16 [==============================] - 118s 7s/step - loss: 1.3016 - accuracy: 0.4782 - val_loss: 1.3486 - val_accuracy: 0.3797
Epoch 3/10
 3/16 [====>.........................] - ETA: 1:35 - loss: 0.9418 - accuracy: 0.5208

In [ ]:
# accuracies
plt.plot(history.history['accuracy'], label='train acc')
plt.plot(history.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()

In [ ]:
# Loss
plt.plot(history.history['loss'], label='train loss')
plt.plot(history.history['val_loss'], label='val loss')
plt.legend()
plt.show()

In [ ]:
model.evaluate(test_x,test_y,batch_size=32)